In [52]:
import torch
import torch.nn as nn
import numpy as np


class LongShortTermMemoryModel(nn.Module):

    def __init__(self, encoding_size):
        super(LongShortTermMemoryModel, self).__init__()

        self.hidden_state = None
        self.cell_state = None
        self.lstm = nn.LSTM(encoding_size, 128)  # 128 is the state size
        self.dense = nn.Linear(128, encoding_size)  # 128 is the state size

    def reset(self):
        zero_state = torch.zeros(1, 1, 128)  # Shape: (number of layers, batch size, state size)
        self.hidden_state = zero_state
        self.cell_state = zero_state


    def logits(self, x):  # x shape: (sequence length, batch size, encoding size)
        out, (self.hidden_state, self.cell_state) = self.lstm(x, (self.hidden_state, self.cell_state))
        return self.dense(out.reshape(-1, 128))

    def f(self, x):  # x shape: (sequence length, batch size, encoding size)
        return torch.softmax(self.logits(x), dim=1)

    def loss(self, x, y):  # x shape: (sequence length, batch size, encoding size), y shape: (sequence length, encoding size)
        return nn.functional.cross_entropy(self.logits(x), y.argmax(1))

In [53]:
index_to_char = [' ', 'h', 'a', 'l', 'o', 't', 'r', 'f', 'c', 'm', 'p', 's', 'n']
char_encodings = np.identity(len(index_to_char), dtype=float).tolist()
encoding_size = len(char_encodings)

emojis = {
    "hat": ["\U0001F3A9", "hat "],
    "rat": ["\U0001F401", "rat "],
    "cat": ["\U0001F408", "cat "],
    "flat": ["\U0001F3E6", "flat"],
    "matt": ["\U0001F468", "matt"],
    "cap": ["\U0001F9E2", "cap "],
    "son": ["\U0001F349", "son "]
}

emojiMatrix = np.eye(len(emojis))

def get_index(y: int):
    i = 0
    for value in emojis.values():
        if i == y:
            return value
        i += 1

letter_dict = {
    index_to_char[i]: i for i in range(0, len(index_to_char))
}

def one_hot(x: str):
    char = char_encodings[letter_dict[x]]
    return char

x_train = torch.tensor([
                       [[one_hot('h')], [one_hot('a')], [one_hot('t')], [one_hot(' ')]],
                       [[one_hot('r')], [one_hot('a')], [one_hot('t')], [one_hot(' ')]],
                       [[one_hot('c')], [one_hot('a')], [one_hot('t')], [one_hot(' ')]],
                       [[one_hot('f')], [one_hot('l')], [one_hot('a')], [one_hot('t')]],
                       [[one_hot('m')], [one_hot('a')], [one_hot('t')], [one_hot('t')]],
                       [[one_hot('c')], [one_hot('a')], [one_hot('p')], [one_hot(' ')]],
                       [[one_hot('s')], [one_hot('o')], [one_hot('n')], [one_hot(' ')]],
                        ], dtype=torch.float)

y_train = torch.tensor([
                        [emojiMatrix[0], emojiMatrix[0], emojiMatrix[0], emojiMatrix[0]],
                        [emojiMatrix[1], emojiMatrix[1], emojiMatrix[1], emojiMatrix[1]],
                        [emojiMatrix[2], emojiMatrix[2], emojiMatrix[2], emojiMatrix[2]],
                        [emojiMatrix[3], emojiMatrix[3], emojiMatrix[3], emojiMatrix[3]],
                        [emojiMatrix[4], emojiMatrix[4], emojiMatrix[4], emojiMatrix[4]],
                        [emojiMatrix[5], emojiMatrix[5], emojiMatrix[5], emojiMatrix[5]],
                        [emojiMatrix[6], emojiMatrix[6], emojiMatrix[6], emojiMatrix[6]]
                       ], dtype=torch.float)


In [54]:
model = LongShortTermMemoryModel(encoding_size)

optimizer = torch.optim.RMSprop(model.parameters(), 0.001)
for epoch in range(500):
    for i in range(x_train.size()[0]):
        model.reset()
        model.loss(x_train[i], y_train[i]).backward()
        optimizer.step()
        optimizer.zero_grad()


def get_emoji(emoji: str):
    y = -1
    model.reset()
    for i in range(len(emoji)):
        char_i = index_to_char.index(emoji[i])
        y = model.f(torch.tensor([[char_encodings[char_i]]]))
    print(get_index(y.argmax(1)))
    

get_emoji("hat")
get_emoji("rt")
get_emoji("matt")
get_emoji("maattt")

['🎩', 'hat ']
['🐁', 'rat ']
['👨', 'matt']
['👨', 'matt']
